# __Long Short Term Memory Decoder__

### __Deep Learning__

#### __Project: Image Captioning with Visual Attention__

In [1]:
import os
os.chdir(os.environ["PYTHONPATH"])

import torch
import matplotlib.pyplot as plt
import seaborn as sns

import scripts.data_loading as dl
import scripts.data_processing as dp
from scripts import model

%matplotlib inline
%load_ext autoreload
%autoreload 2


plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["image.cmap"] = "plasma"

In [2]:
coco_train = dl.CocoCaptions(
    dl.TRAINING_DATASET_PATHS[dl.DatasetType.TRAIN],
    dp.VGGNET_PREPROCESSING_PIPELINE,
    dp.TextPipeline(),
)

loading annotations into memory...
Done (t=0.84s)
creating index...
index created!


In [3]:
coco_loader = dl.CocoLoader(coco_train, batch_size=2, num_workers=1)
it = iter(coco_loader)
image_batch, caption_batch = next(it)

In [4]:
encoder = model.VGG19Encoder()
feature_maps, feature_mean = encoder.forward(image_batch)

In [5]:
decoder = model.LSTMDecoder(
    num_embeddings=len(coco_train.target_transform.vocabulary),
    embedding_dim=8,
    encoder_dim=feature_mean.shape[-1],
    decoder_dim=16,
    attention_dim=4
)

In [6]:
embeddings = decoder.word_embedding(caption_batch)

In [7]:
print(f"One-hot encoded caption shape = {caption_batch[0].shape}")
print(f"Embedding shape = {embeddings[0].shape}")

One-hot encoded caption shape = torch.Size([51])
Embedding shape = torch.Size([51, 8])


In [8]:
print(caption_batch[0])
print(embeddings[0])

tensor([10000,    12,    78,    22,    34,   850,    31,     4,     0,   182,
            1,  1421,    65,     2,     0,     8,   283,  1486,     0,    34,
          204,     9,  1584,    47,    34,   924,     1,    34,  1118,     5,
          133,   167,    62,    34,  1186,     4,   121,     4,    36,     2,
          244,     6,    12,  1186,     1,   206,     6,   314, 10002,   139,
        10001])
tensor([[ 0.6191, -1.3173,  1.2111,  0.6039, -0.8797, -0.1324,  0.5306, -0.3078],
        [ 1.3595,  0.0215,  1.0839,  0.3650, -0.3223,  0.5422,  0.4306,  2.0742],
        [ 1.8461,  0.6147, -0.3262, -0.4988,  0.2438, -0.5828, -0.1015,  0.6469],
        [-0.5625,  1.4439, -0.5091,  0.0516, -1.4999, -1.2306, -0.9444, -0.2195],
        [ 0.2375, -1.1515,  1.4677,  0.2530, -0.9711, -1.4489, -1.1514, -0.0703],
        [-0.8641,  1.3573, -0.6889,  1.8019, -1.1206, -0.0779,  0.5012, -2.1369],
        [ 1.0276,  0.5393, -0.6571, -1.3861, -1.5626,  2.0819,  0.9656, -1.4506],
        [-1.1893,  0

In [9]:
h = decoder.init_h(feature_mean)
c = decoder.init_c(feature_mean)

print(f"Initial h shape = {h.shape}")
print(f"Initial c shape = {c.shape}")

Initial h shape = torch.Size([2, 16])
Initial c shape = torch.Size([2, 16])


In [10]:
# Initial h, c of LSTM computed by MLP(feature_maps_mean)
print(h[0])
print(c[0])

tensor([-0.1561, -0.2910,  0.0597,  0.0636,  0.0212,  0.0774,  0.1135, -0.0091,
        -0.1465,  0.0534, -0.1086, -0.2076, -0.0314,  0.0773, -0.0264, -0.1042],
       grad_fn=<SelectBackward>)
tensor([-1.1120e-01,  3.9517e-01,  2.5227e-01, -1.8984e-04, -1.4602e-01,
        -2.2757e-01, -3.2849e-02,  2.0612e-02, -6.1846e-02, -2.8293e-02,
        -2.4599e-02, -1.2679e-02, -2.5941e-02,  6.1147e-02,  3.0811e-02,
         2.0098e-01], grad_fn=<SelectBackward>)


In [11]:
# How to get word embeddings of words at particular index of a caption in batch
index = 50
embeddings[:, index]

tensor([[ 0.3672,  1.1899,  0.8743,  0.2693, -0.8009,  0.3842, -1.2845, -0.0401],
        [ 0.3672,  1.1899,  0.8743,  0.2693, -0.8009,  0.3842, -1.2845, -0.0401]],
       grad_fn=<SelectBackward>)

In [12]:
feature_mean

tensor([[0.1412, 0.1195, 0.1757, 0.1956, 0.2014, 0.1619, 0.1296, 0.1212, 0.1625,
         0.2444, 0.2788, 0.3305, 0.3097, 0.3634, 0.2017, 0.1454, 0.0949, 0.0646,
         0.0738, 0.0580, 0.0685, 0.0834, 0.1497, 0.3198, 0.2564, 0.2706, 0.2274,
         0.2668, 0.4305, 0.2778, 0.3106, 0.2719, 0.2167, 0.0923, 0.0908, 0.1187,
         0.3236, 0.6264, 0.3948, 0.4090, 0.3418, 0.3622, 0.3230, 0.1912, 0.2707,
         0.2215, 0.1806, 0.0852, 0.0931, 0.1090, 0.2908, 0.4428, 0.1921, 0.2183,
         0.2172, 0.3397, 0.3572, 0.2309, 0.2297, 0.1774, 0.2011, 0.1361, 0.1507,
         0.1538, 0.2483, 0.2350, 0.1305, 0.1665, 0.2978, 0.4957, 0.3393, 0.1802,
         0.1848, 0.1314, 0.2549, 0.3474, 0.3141, 0.2434, 0.2089, 0.1747, 0.2000,
         0.2210, 0.3853, 0.5794, 0.2008, 0.1367, 0.1783, 0.1296, 0.2880, 0.4784,
         0.5109, 0.4420, 0.3456, 0.3144, 0.3219, 0.3269, 0.3614, 0.5044, 0.2522,
         0.1661, 0.2265, 0.1435, 0.2713, 0.4259, 0.4878, 0.4517, 0.3450, 0.2899,
         0.2741, 0.2950, 0.3

In [13]:
# How to concatenate context and embedding word
torch.cat([embeddings[:, index], feature_mean], dim=1)

tensor([[ 0.3672,  1.1899,  0.8743,  0.2693, -0.8009,  0.3842, -1.2845, -0.0401,
          0.1412,  0.1195,  0.1757,  0.1956,  0.2014,  0.1619,  0.1296,  0.1212,
          0.1625,  0.2444,  0.2788,  0.3305,  0.3097,  0.3634,  0.2017,  0.1454,
          0.0949,  0.0646,  0.0738,  0.0580,  0.0685,  0.0834,  0.1497,  0.3198,
          0.2564,  0.2706,  0.2274,  0.2668,  0.4305,  0.2778,  0.3106,  0.2719,
          0.2167,  0.0923,  0.0908,  0.1187,  0.3236,  0.6264,  0.3948,  0.4090,
          0.3418,  0.3622,  0.3230,  0.1912,  0.2707,  0.2215,  0.1806,  0.0852,
          0.0931,  0.1090,  0.2908,  0.4428,  0.1921,  0.2183,  0.2172,  0.3397,
          0.3572,  0.2309,  0.2297,  0.1774,  0.2011,  0.1361,  0.1507,  0.1538,
          0.2483,  0.2350,  0.1305,  0.1665,  0.2978,  0.4957,  0.3393,  0.1802,
          0.1848,  0.1314,  0.2549,  0.3474,  0.3141,  0.2434,  0.2089,  0.1747,
          0.2000,  0.2210,  0.3853,  0.5794,  0.2008,  0.1367,  0.1783,  0.1296,
          0.2880,  0.4784,  

In [14]:
predictions, attention_scores = decoder.forward(feature_maps, feature_mean, caption_batch)

In [15]:
caption_batch.shape

torch.Size([2, 51])

Number of predicted words == caption_len - 1

In [16]:
print(predictions.shape)

torch.Size([50, 2, 10004])


Random guess prediction probability

In [17]:
predictions[0, 0, :100]

tensor([ 0.4375, -0.6261,  0.5317, -0.0880,  0.4891,  0.2925,  0.3068, -0.6892,
         0.3137,  0.2884, -0.0486,  0.7690,  0.3855, -0.2923,  1.2620, -0.1417,
        -0.0025, -0.6173, -0.5011,  0.2308, -0.0523,  0.3094, -0.0233,  0.1693,
         1.2063, -0.1480, -0.3786, -0.8453, -0.3958,  0.3300, -0.9954,  0.8700,
         0.4820,  0.6700,  0.3797,  0.5473,  0.0105,  0.1244,  0.8294, -0.4222,
        -0.6834, -0.1532, -0.7727, -0.4052,  0.2997,  0.3803, -0.3370,  0.2397,
        -0.9334, -0.0588, -0.5762, -0.8898, -1.3642, -0.2172,  1.0626,  0.8194,
         0.3410,  0.1657, -0.8686,  1.3753, -0.1424, -0.8258,  0.2633,  0.0301,
         0.3335, -0.0935, -0.3745, -0.1877, -0.3575, -0.1304,  0.7790,  0.2605,
        -0.2786, -0.5751, -0.1760,  0.5945, -0.4125, -0.3128,  0.4323, -0.2261,
        -0.0516,  0.2576, -0.4456, -1.1882,  0.1064,  0.7113, -1.1081, -0.0718,
        -0.4187, -0.2525,  0.1215,  0.2907, -0.5742,  0.3292, -1.0017, -0.1348,
        -0.8455, -0.3773, -0.2972,  0.50

In [18]:
1 / 10_004

9.996001599360256e-05

Attention_scores

In [19]:
attention_scores.shape

torch.Size([50, 2, 512])

In [20]:
attention_scores[0, 0]

tensor([0.0021, 0.0036, 0.0020, 0.0013, 0.0017, 0.0021, 0.0020, 0.0020, 0.0021,
        0.0015, 0.0020, 0.0022, 0.0020, 0.0021, 0.0016, 0.0020, 0.0030, 0.0022,
        0.0020, 0.0020, 0.0015, 0.0020, 0.0028, 0.0014, 0.0020, 0.0020, 0.0019,
        0.0014, 0.0024, 0.0020, 0.0020, 0.0020, 0.0020, 0.0019, 0.0020, 0.0016,
        0.0016, 0.0020, 0.0020, 0.0020, 0.0020, 0.0019, 0.0021, 0.0019, 0.0019,
        0.0020, 0.0013, 0.0017, 0.0022, 0.0020, 0.0020, 0.0016, 0.0020, 0.0021,
        0.0020, 0.0019, 0.0017, 0.0020, 0.0019, 0.0020, 0.0019, 0.0021, 0.0019,
        0.0019, 0.0021, 0.0018, 0.0020, 0.0020, 0.0018, 0.0018, 0.0017, 0.0020,
        0.0019, 0.0018, 0.0019, 0.0013, 0.0018, 0.0020, 0.0017, 0.0022, 0.0019,
        0.0019, 0.0020, 0.0022, 0.0021, 0.0020, 0.0020, 0.0018, 0.0021, 0.0020,
        0.0016, 0.0020, 0.0019, 0.0023, 0.0019, 0.0020, 0.0016, 0.0023, 0.0022,
        0.0024, 0.0022, 0.0021, 0.0020, 0.0022, 0.0020, 0.0020, 0.0018, 0.0020,
        0.0025, 0.0020, 0.0020, 0.0020, 